This program uses the iris dataset, which comes included with the sklearn package, to construct a decision tree which is able to identify the class of a flower based on attributes such as its petal length and width.

A decision tree works by asking a question at each layer, and following the appropriate path based on the answer to the question.

![A sample decision tree.](./decision_tree_example.png)

For example, if the given conditions were {'windy': True, 'outlook': 'Rain', 'humidity': 72, 'temperature': 28c}, following the decision tree would lead us to the output "No".

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
data = datasets.load_iris()
x = data.data
y = data.target

Split the dataset into four parts: two for training, two for testing. The training and testing sets are further divided into inputs (x) and outputs (y).

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 99) # random_state is the seed for the random number generator

Now we train our DecisionTreeClassifier on its default hyperparameters, save for the random_state, which is set to a known value for the purposes of replicating results.

In [4]:
clf = DecisionTreeClassifier(random_state=1)
clf.fit(x_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,1
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


With a trained DecisionTreeClassifier, we test it on the data kept aside for testing purposes.

In [5]:
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred) # accuracy_score = number of correctly classified samples / total number of samples
print(f'Base accuracy: {round(accuracy*100, 2)}%')

Base accuracy: 95.56%


Now that we have a working classifier, it is time to find the best hyperparameters for it. The hyperparameters are the parameters relating to the classifier itself. Optimizing these will lead to better performance and results.

In [6]:
param_grid = {
    'max_depth': range(1, 10, 1),
    'min_samples_leaf': range(1, 20, 2),
    'min_samples_split': range(2, 20, 2),
    'criterion': ["entropy", "gini"]
} # param_grid defines the possible values each hyperparameter can take

grid_search = GridSearchCV(estimator= clf, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(x_train, y_train)

print(f"Best accuracy: {round(grid_search.best_score_*100, 2)}%")
print(f"Best accuracy better than base accuracy? {grid_search.best_score_ > accuracy}.")
print(f"Best hyperparameters: {grid_search.best_estimator_}")

if grid_search.best_score_ > accuracy:
    print("Since higher accuracy is found, updating hyperparameters.")
    optimized_tree_clf = grid_search.best_estimator_
else:
    print("Since higher accuracy is not found, hyperparameters remain same.")
    optimized_tree_clf = clf

Fitting 5 folds for each of 1620 candidates, totalling 8100 fits
Best accuracy: 97.14%
Best accuracy better than base accuracy? True.
Best hyperparameters: DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=3,
                       random_state=1)
Since higher accuracy is found, updating hyperparameters.


The below code creates a decision tree using the optimized classifier.

In [7]:
plt.figure(figsize = (100, 100))
plot_tree(optimized_tree_clf, filled = True, feature_names = data.feature_names, class_names = data.target_names)
plt.show()

And here is a simplified version of the decision tree, showing only the conditions and the classes.

In [10]:
fig, ax = plt.subplots(figsize = (100, 100))
plot_tree(optimized_tree_clf, filled = True, ax = ax, feature_names = data.feature_names, class_names = data.target_names)
def replace_text(obj):
    if type(obj) == matplotlib.text.Annotation:
        txt = obj.get_text()
        txt = re.sub("\nsamples[^$]*class","\nclass",txt)
        txt = re.sub("\nentropy[^$]*class","\nclass",txt)
        obj.set_text(txt)
    return obj
    
ax.properties()['children'] = [replace_text(i) for i in ax.properties()['children']]

Finally, the below code shows how to make our own predictions with our own data that is not taken from the prepared dataset.

In [11]:
example_input = np.array([6.7, 2.9, 6.1, 2.1]) # ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
example_prediction = optimized_tree_clf.predict(example_input.reshape(1, -1)) # reshaping array as classifier expects 2D array, using (1, -1) results in a 2D matrix-like array with just 1 row
print(f"The flower with the dimensions {example_input} belongs to the class {data.target_names[example_prediction]}") # examplePrediction returns index of class in data.target_names

The flower with the dimensions [6.7 2.9 6.1 2.1] belongs to the class ['virginica']
